# 
# <center> CBERS-4A IMAGERY FOR MAPPING URBAN LAND COVER IN THE AMAZON  </center>

<br/>

<div style="text-align: center;font-size: 90%;">
    Bruno Dias dos Santos, Michelle Azevedo, Luisa Akemi, Carolina Moutinho Duque de Pinho, Antonio Paez and Silvana Amaral  
    <br/><br/>
    National Institute for Space Research (INPE)
    <br/>
    Avenida dos Astronautas, 1758, Jardim da Granja, São José dos Campos, SP 12227-010, Brazil
    <br/><br/>
    e-mail to: <div><a href="mailto:bruno.santos@inpe.br">bruno.santos@inpe.br</a></div>
    <br/><br/>
</div>
<br/>
<div style="text-align: justify;  margin-left: 25%; margin-right: 25%;">
<b>Abstract.</b> The Brazilian Institute of Geography and Statistics (IBGE) identified Intraurban Typologies in several Brazilian urban concentrations using microdata from the 2010 Census. However, using weighting areas as a spatial unit of analysis and the indicators adopted makes it challenging to replicate the methodology for other areas. This paper aims to adapt the IBGE study to the reality of an Amazonian city, choosing Santarém as study site. To achieve this objective, we adapted the socioeconomic indicators to the Amazonian context. As a result, we identified  six intra-urban typologies through an unsupervised classification, which differ concerning  population profile, housing conditions and location in the study area.
    
Key words — Land cover, GEOBIA, Random Forest, CBERS-4A, Data Mining.

</div>    

<br/>

### 1º Etapa: Tratamento e seleção de variáveis

Importing libraries:

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math
import folium
import mapclassify
import geopandas as gpd
import shapely
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score,ConfusionMatrixDisplay,classification_report,confusion_matrix

Reading the segments with attributes and creating a GeoDataFrame:

In [ ]:
obj = gpd.read_file("E:\\00_INPE\\PDI\\TRABALHO-FINAL\\cameta\\FINAIS\\classificado\\nivel1_cameta.shp")

In [ ]:
obj

Defining the identifier variable of each feature:

In [ ]:
indice = 'DN'

Defining the variable with the sample classes. This variable must be categorical or discrete:

In [ ]:
TARGET = 'TARGET'

target = []
cont = 0

for classe in obj[TARGET].unique():
    if not pd.isnull(classe):
        target.append(classe)

target

Copying the 'index' and 'TARGET' information from the original base to a new GeoDataFrame:

In [ ]:
geom = obj[[indice,'geometry']]

In [ ]:
geom

Extracting categorical and numerical variables for the data treatment:

In [ ]:
var_num = pd.DataFrame(obj.select_dtypes(include=['float64','int64','int','float']))
var_cat = pd.DataFrame(obj.select_dtypes(include=['string','object']))

In [ ]:
var_cat

In [ ]:
var_num

Removing the sample column for the data treatment:

In [ ]:
try: 
    var_cat = var_cat.drop(columns = TARGET)
except:
    var_num = var_num.drop(columns = TARGET)

Removendo outliers dos campos numéricos:

In [ ]:
#Function for removing outliers by considering as outlier values greater than 2.698 σ (standard deviation) of the normal distribution curve: 

def rmv_outliers(DataFrame, col_name):
    intervalo = 2.698 * DataFrame[col_name].std()
    media = DataFrame[col_name].mean()
    DataFrame.loc[DataFrame[col_name] < (media - intervalo), col_name] = np.nan
    DataFrame.loc[DataFrame[col_name] > (media + intervalo), col_name] = np.nan

for coluna in var_num.columns:
    rmv_outliers(var_num, coluna)

Normalizing and filling empty values in numeric fields:

In [ ]:
#dummy = var_num.iloc[:,1:].mean()
dummy = 0

var_num.iloc[:,1:] = var_num.iloc[:,1:].fillna(dummy)

var_num.iloc[:,1:] =(var_num.iloc[:,1:] - var_num.iloc[:,1:].min())/(var_num.iloc[:,1:].max() - var_num.iloc[:,1:].min())
var_num

Applying OneHotEncoder to variables of a categorical type and creating a DataFrame of the data:

In [ ]:
aux = obj[[indice, TARGET]]

try:
    var_cat = pd.get_dummies(var_cat[:-1], drop_first=True)
    obj = (aux.merge(var_num, left_on=indice, right_on=indice)).merge(var_cat, left_index=True, right_index=True)
    
except:
    obj = (aux.merge(var_num, left_on=indice, right_on=indice))
    print("Não há variáveis categóricas para aplicar OneHotEncode")

obj

Viewing descriptive statistics of the already processed database:

In [ ]:
obj.describe()

Creating a copy of the already processed DataFrame:

In [ ]:
saida =  obj
saida

Selecting only the sampled features to extract the statistics:

In [ ]:
obj = obj[obj[TARGET].isin(target)]
obj

Calculate the R² of the Anova of each column against the Target column:

In [ ]:
iv = {}

for coluna in obj.columns:  
    if coluna != TARGET:
        counts = obj.groupby(TARGET, sort=True)[coluna].count() # Count of elements in each sample class
        medias = obj.groupby(TARGET, sort=True)[coluna].mean() # Average of each caluna per sample class 
        aux = 0        
        for i in range(len(counts)):
            try:
                aux = aux + counts.iloc[i]*((medias.iloc[i] - obj[coluna].mean())**2)
            except:
                aux = 0
        
        if (sum(counts))*((obj[coluna].std())**2) == 0:
            iv[coluna] = aux/0.00001
        else:                
            iv[coluna] = aux/((sum(counts))*((obj[coluna].std())**2))
        
        print("Rodou: ", coluna)

iv = sorted(iv.items(), key=lambda x: x[1], reverse=True)

In [ ]:
iv

Bloxpot plot of the variable with the best explaining power in relation to the sample classes:

In [ ]:
plt.figure(figsize=(15,5))
sns.boxplot(x=TARGET , y= obj[iv[0][0]], order= obj[TARGET].sort_values().unique(), data = obj)

Bloxpot plot of the variable with the worst explaining power in relation to the sample classes:

In [ ]:
plt.figure(figsize=(15,5))
sns.boxplot(x=TARGET , y= obj[iv[-1][0]], order= obj[TARGET].sort_values().unique(), data = obj)

Removing poorly explanatory variables from a threshold and variables with possible 'NaN' values:

In [ ]:
lim_min = 0.1 #Alterar limite inferior para remoção das variáveis
aux = []

print(f'Total de variáveis antes da remoção: {len(iv)}')

for i in range(len(iv)):
    if math.isnan(iv[i][1]):
        aux.append(iv[i])
    elif iv[i][0] != indice:
        if iv[i][1] < lim_min:
            aux.append(iv[i])

for i in aux:
    iv.remove(i)

print(f'Total de variáveis depois da remoção: {len(iv)}')

Viewing the most explanatory variables:

In [ ]:
iv

Defining the correlation factor to be considered.

Given two variables [i,j], the correlation between i and j will be calculated. If the correlation between the two variables is greater than the maximum correlation factor, we will exclude the one with less explanatory power from its R²:

In [ ]:
fator = 0.70

Removal of highly correlated _(Pearson)_ variables:

In [ ]:
colunas = []
aux = []

for i, j in iv:
    colunas.append(i)
    aux.append(i)

for i in range(len(colunas)):
    for j in range(len(colunas)):
        if j > i and abs(saida[colunas[i]].corr(saida[colunas[j]])) > fator:
            if (colunas[j] in aux) and (colunas[j] != indice):
                aux.remove(colunas[j])

Output Dataframe variables:

In [ ]:
len(aux)

In [ ]:
aux

Visualizing the correlation between variables:

In [ ]:
corr_df = saida[aux].corr()

corr_df.style.background_gradient(cmap='Spectral')

In [ ]:
corr_df = saida[aux].corr(method='pearson')

plt.figure(figsize=(10, 10))
sns.heatmap(corr_df, annot=False)
plt.show()

In [ ]:
aux.append(TARGET)

Saving a shapefile with the treated data:

In [ ]:
geom = geom.merge(saida[aux], left_on=indice, right_on=indice)
geom

In [ ]:
geom.to_file("E:\\00_INPE\\0_DISSERTACAO\\VALIDACAO\\COBERTURA\\segmentos_cobertura_tratados.shp")

### Supervision Classification of the land cover

Selecting only the database with samples to build the supervised classification model:

In [ ]:
amostras = geom.replace(to_replace='None', value=np.nan).dropna()
amostras

Separating the training and validation data:

In [ ]:
X = pd.DataFrame(amostras.iloc[:,2:-1])
Y = pd.DataFrame(amostras[TARGET]).to_numpy()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, stratify = Y, random_state=42)

In [ ]:
x_train.shape

In [ ]:
y_train.shape

In [ ]:
x_test.shape

In [ ]:
y_test.shape

Visualizing the sample classes:

In [ ]:
amostras[TARGET].unique()

In [ ]:
amostras.groupby(TARGET)[indice].nunique()

Selection of hyperparameters and values for RandomizedSearchCV:

In [ ]:
parametros = {'n_estimators':[1,20,50,100,150,200,250,300,350,400,450,500,550,600,700,800,900,1000,1500,2000],
              'criterion':['gini','entropy'],
              'max_depth':[5,10,20, None],
              'min_samples_split':[2,5,10],
              'min_samples_leaf': [1, 2, 4],
              'bootstrap': [True, False]}

Defining the RandomizedSearchCV:

In [ ]:
modelo = RandomizedSearchCV(estimator = RandomForestClassifier(), n_iter = 100, verbose=2, random_state=42, param_distributions = parametros, scoring='f1_macro', n_jobs=-1, cv=5)

Designing the Random Forest classification model:

In [ ]:
modelo.fit(x_train, y_train)

Visualizing the best combination of hyperparameters:

In [ ]:
modelo.best_params_

In [ ]:
modelo.best_score_

In [ ]:
modelo.best_estimator_

Using the trained model to predict the validation database:

In [ ]:
y_pred = modelo.predict(x_test)

Viewing the most used variables in the Random Forest model:

In [ ]:
plt.barh(x_test.columns, modelo.best_estimator_.feature_importances_ )

Viewing performance metrics of the classification model:

In [ ]:
macro = f1_score(y_test, y_pred, average = 'macro')
wei = f1_score(y_test, y_pred, average = 'weighted')
accuracy = accuracy_score(y_test, y_pred)

results = {'F1_Score_Macro': macro,
             'F1_Score_Weighted': wei,
             'Global Acuraccy': accuracy 
            }

pd.DataFrame.from_dict(results, orient='index', dtype=None, columns=['Métricas'])

Confusion matrix:

In [ ]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [ ]:
data = {'Reference': y_test.flatten(), 'Predicted': y_pred}
df = pd.DataFrame(data, columns = ['Reference','Predicted'])
mc = pd.crosstab(df['Reference'], df['Predicted'], rownames=['Reference'], colnames=['Predicted'])

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(mc, annot=True,fmt='g',  cmap = 'YlGnBu',linewidths=.5)
plt.title('Matriz de Confusão', fontweight='bold', fontsize=14)
plt.xlabel('Classificação', fontsize=12)
plt.ylabel('Referência',fontsize=12)
plt.show()

In [ ]:
classe = []
Accuracy = []
Precision = []
Recall = []
F1_Score = []

for i in range(mc.shape[0]):
    TP = mc.iloc[i,i]
    FP = mc.iloc[i,:].sum() - TP
    FN = mc.iloc[:,i].sum() - TP
    TN = mc.sum().sum()-TP-FP-FN
    
    classe.append(mc.index[i]) 
    Accuracy.append((TP+TN)/mc.sum().sum())
    Precision.append(TP/(TP+FP))
    Recall.append(TP/(TP+FN))
    F1_Score.append(((2*Precision[i]*Recall[i])/(Precision[i] + Recall[i])))
    

avaliacao = {'classe': classe,
            'Precision': Precision,
             'Recall': Recall,
             'F1_Score':F1_Score,
             'Acuraccy':Accuracy
            }
       
pd.DataFrame(avaliacao)

Inserting the classification into the database shapefile:

In [ ]:
geom['CLASSIFICACAO'] = modelo.predict(geom.iloc[:,2:-1])

Land cover classification map:

In [ ]:
geom.plot(column ='CLASSIFICACAO', legend=True, cmap = 'tab20', categorical=True, legend_kwds={'loc': 'center left', 'bbox_to_anchor':(1,0.5)})

In [ ]:
geom.explore(column="CLASSIFICACAO", tooltip="CLASSIFICACAO",tiles="CartoDB positron",
             categorical = True, cmap='Set2', style_kwds=dict(color="grey", weight=0.01))

Exporting the shapefile with classification:

In [ ]:
geom.to_file("E:\\00_INPE\\0_DISSERTACAO\\VALIDACAO\\COBERTURA\\cobertura_cameta_classificado.shp")